# 2: Syllabification
- Takes the WAV datasets generated by '1.0-segment-song-from-wav-files' and segments into spectograms of syllables.
- Outputs HDF5 file containing metadata about the cowbird singing, the syllables sung, the length of the syllables, and the file of origin.

## Imports

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import pandas as pd
from datetime import datetime, timedelta
import os
import sys
import IPython.display
import pickle
from sklearn.externals.joblib import Parallel, delayed
from PIL import Image

import warnings 
warnings.filterwarnings('ignore')

In [26]:
from avgn.segment_song.preprocessing import *
import avgn.segment_song.wav_to_syllables as w2s
import avgn.spectrogramming.spectrogramming as sg

## Parameters for Syllable Segmentation
- Set to brown-headed cowbird recordings.
- **TODO: UPDATE PARAMETERS**

In [34]:
syll_size = 128
hparams = {
    'species':'cb',
    # filtering
    'highcut':15000,
    'lowcut':0,
    
    # spectrograms
    'mel_filter': True, # should a mel filter be used?
    'num_mels':syll_size, # how many channels to use in the mel-spectrogram
    'num_freq':512, # how many channels to use in a spectrogram 
    'num_freq_final': syll_size, # how many channels to use in the resized spectrogram
    'sample_rate':44100, # what rate are your WAVs sampled at?
    'preemphasis':0.97, 
    'min_silence_for_spec': 0.5, #minimum length of silence for a spectrogram to be considered a good spectrogram
    'frame_shift_ms':0.5, # step size for fft
    'frame_length_ms':6, # frame length for fft
    'min_level_db':-95, # minimum threshold db for computing spe 
    'spec_thresh_min': -40, # (db)
    'spec_thresh_delta': 5, # (db) what 
    'ref_level_db':20, # reference db for computing spec
    'sample_rate':44100, # sample rate of your data
    'fmin': 300, # low frequency cutoff for mel filter
    'fmax': None, # high frequency cutoff for mel filter
    
    # Vocal Envelope
    'smoothing' : 'gaussian', # 'none', 
    'envelope_signal' : "spectrogram", # spectrogram or waveform, what to get the vocal envelope from
    'gauss_sigma_s': .0001,
    'FOI_min': 4, # minimum frequency of interest for vocal envelope (in terms of mel)
    'FOI_max': 24, # maximum frequency of interest for vocal envelope (in terms of mel)
    
    # Silence Thresholding
    'silence_threshold' : 0, # normalized threshold for silence
    'min_len' : 5., # minimum length for a vocalization (fft frames)
    'power_thresh': .3, # Threshold for which a syllable is considered to be quiet weak and is probably noise

    # Syllabification
    'min_syll_len_s' : 0.03, # minimum length for a syllable
    'segmentation_rate': 0.0,#0.125, # rate at which to dynamically raise the segmentation threshold (ensure short syllables)
    'threshold_max': 0.25,
    'min_num_sylls': 20, # min number of syllables to be considered a bout
    'slow_threshold':0.0,#0.02, # second slower threshold
    'max_size_syll': syll_size, # the size of the syllable
    'resize_samp_fr': int(syll_size*5.0), # (frames/s) the framerate of the syllable (in compressed spectrogram time components)
    
    # Sencond pass syllabification
    'second_pass_threshold_repeats':50, # the number of times to repeat the second pass threshold
    'ebr_min': 0.05, # expected syllabic rate (/s) low 
    'ebr_max':  0.2, # expected syllabic rate (/s) high 
    'max_thresh':  0.02, # maximum pct of syllabic envelope to threshold at in second pass
    'thresh_delta':  0.005, # delta change in threshold to match second pass syllabification
    'slow_threshold': 0.005, # starting threshold for second pass syllabification
    
    'pad_length' : syll_size, # length to pad spectrograms to 
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':60,
    'power':1.5,

    # thresholding out noise
    'mel_noise_filt' : 0.15, # thresholds out low power noise in the spectrum - higher numbers will diminish inversion quality
}

globals().update(hparams)
now_string = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # this is used to identify this training instance

# save the dictionary so that we can reload it for recovering waveforms
dict_save = '../data/parameter_dictionaries/'+now_string+'_dict.pickle'
with open(dict_save, 'wb') as f:
    pickle.dump(hparams, f, protocol=pickle.HIGHEST_PROTOCOL)
print(dict_save)

FileNotFoundError: [Errno 2] No such file or directory: '../data/parameter_dictionaries/2019-08-12_15-17-23_dict.pickle'

## Visualization
- Visualize segmentation algorithm on an example WAV file (ie. test).
- Read data and runs bandpass filter on data.
- Outputs vizualization.

In [29]:
wav_loc = '../data/cb_wavs/test_bird.wav'

In [ ]:
# read data
csv_loc = '/'.join(wav_loc.split('/')[:-2] + ['csv'] + [wav_loc.split('/')[-1][:-4] + '.csv'])
rate, data= wavfile.read(wav_loc)
(bird,original_wav, start_time) = pd.read_csv(csv_loc, header=None).values[0]
start_time = datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S-%f")

In [ ]:
# bandpass filter (ie. frequency ranges)
data = butter_bandpass_filter(data, lowcut, highcut, rate, order=2).astype('float32')

In [ ]:
print('Rate: ', rate, 'Time sung: ',start_time, 'Length:', len(data)/float(rate))

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))
plt.plot(data, color='black')
ax.set_xlim((0, len(data)))
ax.axis('off')
plt.show()

### Play Song Sample

In [ ]:
# play back a sample of the song
IPython.display.Audio(data=data[:rate*10], rate=44100)

### Generate Spectrograms
- Also outputs vocal envelope of spectogram.
- Outputs visualization and statistics of the generated file.

In [ ]:
_mel_basis = sg._build_mel_basis(hparams) # build a basis function if you are using a mel spectrogram
# Generates the spectrogram and also thresholds out bad spectrograms (e.g. too noisy) - take a look at wav_to_syllables.py to determine if you want this
spec, vocal_envelope, cur_spec_thresh, fft_time_idx, fft_rate = w2s.compute_spec_and_env((data/32768.).astype('float32'), start_time, hparams, rate, _mel_basis, verbose = True, mel_filter=mel_filter)

In [ ]:
# visualize the generated spectrogram
plt.clf()
start_time = fft_time_idx[0]
stop_time = fft_time_idx[-1]
fig, ax = plt.subplots(nrows=2,ncols=1, figsize=(80,6))

ax[0].matshow(spec, interpolation=None, aspect='auto', # cmap=plt.cm.gray_r,
                 cmap=plt.cm.viridis, origin='lower', extent=[start_time,stop_time,0,rate/2])
ax[0].set_title('Spectrogram')

ax[1].plot(vocal_envelope, color = 'red')
ax[1].set_xlim([0,len(vocal_envelope)])
ax[1].set_title('Vocal Envelope')
plt.show()

In [ ]:
# Output some stats of our file
print(''.join(['new Spectrogram Size: ',str(np.shape(spec))]))
print(''.join(['Original Waveform Size: ',str(np.shape(data))]))
print(''.join(['Length (s): ', str(len(data)/float(rate))]))
print(''.join(['Original Sampling Rate (ms) : ', str(1./float(rate))]))
print(''.join(['New Sampling Rate (ms): ', str(round((len(data)/float(rate))*1000 / float(np.shape(spec)[1]), 3))]))

### Detect Onsets/Offsets of Noise/Vocalization
- Of Vocal Envelope, for syllabification.

In [ ]:
# Detect onsets and offsets of vocal envelope
# vocal_envelope =np.mean(norm(wav_spectrogram.T), axis =0)
# vocal_envelope = norm(np.sum(spec, axis = 0))
onsets, offsets = detect_onsets_offsets(vocal_envelope, 
      threshold = silence_threshold,
      min_distance = 0.
     )

print(onsets[0:10], offsets[0:10], len(onsets))

In [ ]:
new_fft_rate = len(data)/rate/float(np.shape(spec)[1])
for i in range(second_pass_threshold_repeats):
    onsets, offsets = w2s.second_pass_threshold(onsets, offsets, vocal_envelope,new_fft_rate, hparams)
    #print(len(onsets))

## Syllabification
- Based on onsests and offsets.
- Plot in histogram and remove short syllables.

In [ ]:
# segment into syllables based upon onset/offsets
all_syllables, all_syllables_time_idx, syll_start = w2s.cut_syllables(onsets, offsets, spec, fft_time_idx, hparams)
# Timing for when each syllable started (seconds)
print(syll_start[0:5])

In [ ]:
w2s.plot_seg_spec(np.concatenate(np.array((onsets,offsets))), spec, fft_time_idx, vocal_envelope, 0, hparams)  

In [ ]:
syllable_lengths = [np.shape(all_syllables[i])[1]/fft_rate for i in range(len(all_syllables))]
print(len(syllable_lengths))

In [ ]:
# Plot how we've thresholded out syllables which are too short to be considered song
print(np.min(syllable_lengths), np.max(syllable_lengths), np.mean(syllable_lengths))
print(syllable_lengths[0:5])
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(16,3))
_ = plt.hist(syllable_lengths,bins=50)
ax.axvline(x=hparams['min_syll_len_s'], color = 'red')
plt.show()

In [ ]:
# remove short syllables
all_syllables, all_syllables_time_idx, syll_start = w2s.threshold_syllables(all_syllables, all_syllables_time_idx, syll_start, min_syll_len_s, fft_rate, power_thresh = power_thresh)

## Test Spectral Reconstruction on Syllable

In [ ]:
syll_num = 10
syll = all_syllables[syll_num]
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(4,4))
ax.matshow(syll, origin='lower', aspect='auto')
plt.show()
print(np.shape(all_syllables[syll_num]))

In [ ]:
mel_inversion_filter = (_mel_basis.T / _mel_basis.sum(axis=1))
mel_inversion_filter =np.nan_to_num(np.transpose(mel_inversion_filter / mel_inversion_filter.sum(axis=1)[:, np.newaxis]))

In [ ]:
# original wav
IPython.display.Audio(data=data[int(all_syllables_time_idx[syll_num][0]):int(all_syllables_time_idx[syll_num][-1])], rate=44100)

In [ ]:
if mel_filter: 
    syll = np.dot(syll.T, mel_inversion_filter).T

In [ ]:
def rescale(X, out_min, out_max):
    return out_min + (X - np.min(X)) * ((out_max - out_min) / (np.max(X) - np.min(X)))

In [ ]:
### adding some noise tends to improve reconstruction quality
syll = rescale(syll, 0,1) +np.reshape(np.random.rand(np.prod(np.shape(syll)))*.25, np.shape(syll))

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(4,4))
ax.matshow(syll, origin='lower', aspect='auto')
plt.show()
print(np.shape(syll))

In [ ]:
# test spectrogram inversion
waveform = sg.inv_spectrogram(rescale(syll,.5,1), hparams)

In [ ]:
# play back a sample of the song
IPython.display.Audio(data=waveform, rate=44100)

### Resize and Pad Spectrograms

In [ ]:
import copy

In [ ]:
all_syllables_comp = copy.deepcopy(all_syllables)

In [ ]:
def sub_mode(syll):
    freqs, bins = np.histogram(syll, bins = 100)
    syll = syll - bins[np.argsort(freqs)][-1]#np.median(syll)#
    syll[syll <0] = 0
    return syll

In [ ]:
# resize spectrogram
all_syllables_comp = w2s.resize_spectrograms(all_syllables_comp, max_size = max_size_syll, resize_samp_fr = resize_samp_fr, fft_rate = fft_rate, n_freq= num_freq_final, pad_length=pad_length)

In [ ]:
# set mode to 0
#all_syllables_comp = [sub_mode(syll) for syll in all_syllables_comp]

In [ ]:

# set mode to 0
all_syllables_comp = [sub_mode(syll) for syll in all_syllables_comp]

# 0 pade
all_syllables_comp = np.array([w2s.pad_spectrogram(i, pad_length) for i in all_syllables_comp])
all_syllables_comp = [(norm(i)*255).astype('uint8') for i in all_syllables_comp]

#### Test spectral reconstruction on syllable

In [ ]:
syll = all_syllables_comp[syll_num]

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(6,3))
_ = ax[0].hist(syll.flatten())
ax[1].matshow(norm(syll.astype('float32')), origin='lower', aspect='auto')
plt.show()

In [ ]:
# remove edges
syll_mean = np.mean(syll, axis=0)
if syll_mean[0] == syll_mean[-1]:
    sb = [i for i in np.arange(1, len(syll_mean)) if (syll_mean[i] == syll_mean[0]) and (syll_mean[i-1] == syll_mean[0])]
    sb = [i for i in sb if i+1 not in sb][0], [i for i in sb if i-1 not in sb][-1]
    syll = syll[:,sb[0]+1:sb[1]-1]  

In [ ]:
if mel_filter:
    resize_shape = (int((np.shape(syll)[1]/hparams['resize_samp_fr']) * (1000/hparams['frame_shift_ms'])), hparams['num_freq_final'])
    syll = np.array(Image.fromarray(np.squeeze(syll)).resize(resize_shape, Image.ANTIALIAS))
    syll = np.dot(syll.T, mel_inversion_filter).T
else:
    resize_shape = (int((np.shape(syll)[1]/hparams['resize_samp_fr']) * (1000/hparams['frame_shift_ms'])), hparams['num_freq'])
    syll = np.array(Image.fromarray(np.squeeze(syll)).resize(resize_shape, Image.ANTIALIAS))

In [ ]:
### adding some noise tends to improve reconstruction quality
syll = rescale(syll, .25,1) +np.reshape(np.random.rand(np.prod(np.shape(syll)))*.25, np.shape(syll))

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(6,3))
_ = ax[0].hist(syll.flatten())
ax[1].matshow(norm(syll.astype('float32')), origin='lower', aspect='auto')
plt.show()

In [ ]:
# test spectrogram inversion
waveform = sg.inv_spectrogram(rescale(syll, .5,1), hparams)

In [ ]:
# play back a sample of the song
IPython.display.Audio(data=waveform, rate=44100)

### Plot computed syllables

In [ ]:
w2s.plt_all_syllables(all_syllables_comp, num_freq_final,  max_rows = 8, max_sylls = 100, width = 1400)
plt.show()

In [ ]:
# This takes a while to run... we're visualizing the whole pipeline on this song
w2s.plot_pipeline(data, vocal_envelope,spec.T,onsets,offsets, all_syllables_comp,rate,all_syllables_time_idx,syllable_lengths, figsize = (100,10))

## Processing data

#### Run through in parallel

In [ ]:
# find the data bird folders
dataset_location = '../../../data/bf_wav/*'
bird_folders = glob(dataset_location)
bird_folders

In [ ]:
data_output_location = '../../../data/bf_song_syllables/'

In [ ]:
skip_existing = False # skip already segmented birds 
parallel = True # run through WAVs in parallel
n_jobs = 10 # how many threads to use if parallel is true
verbosity = 5 # how verbose to make the output of the parallelization (higher = more, 0 = none, >50 output is sent to std.out)
nex = 10 # how many example wavs to plot
save=True
visualize = False # visualize the output of the algorithm for optimizing parameters
if visualize==True: 
    save=False

In [ ]:
import hdbscan 
from skimage.transform import resize
import umap
import time 
import seaborn as sns

def plot_with_labels(data, labels, title = '', ax = None, figsize = (9,9)):
    palette = sns.color_palette('husl', len(np.unique(labels)))
    labs_to_numbers_dict = {l:i for i,l in enumerate(np.unique(labels))}
    np.random.shuffle(palette)
    colors = [palette[labs_to_numbers_dict[x]] if x >= 0 else (0.75, 0.75, 0.75) for x in np.array(labels)]

    if not ax: fig, ax= plt.subplots(nrows=1,ncols=1,figsize=figsize)
    ax.scatter(data.T[0], data.T[1],
               color=colors, alpha = 1, linewidth= 0, s=5)
    ax.axis('off')
    ax.set_title(title)
    if not ax: plt.show()
        
def cluster_data(data, algorithm, args, kwds, verbose = True):
    """ Cluster data using arbitrary clustering algoritm in sklearn
    """
    # Function modified from HDBSCAN python package website
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    if verbose: print('Clustering took {:.2f} s'.format(end_time - start_time))
    return labels

In [ ]:
key_list = (
            'all_bird_wav_file', # Wav file (bout_raw) that the syllable came from
            'all_bird_syll', # spectrogram of syllable
            'all_bird_syll_start', # time that this syllable occured
            'all_bird_t_rel_to_file', # time relative to bout file that this 
            'all_bird_syll_lengths' # length of the syllable
           ) 

for bird_folder in tqdm(bird_folders):
    bird_name = bird_folder.split('/')[-1]
    print(bird_name)

    # prepare the data folder
    if not os.path.exists(''.join([data_output_location,species,'/'])):
        os.makedirs(''.join([data_output_location,species,'/'])) 
    output_filename = ''.join([data_output_location,species,'/',bird_name,'_'+str(syll_size)+'.hdf5'])
    if os.path.exists(output_filename) and skip_existing:
        print('%s already complete, skipping' % (bird_name))
        continue
    
    
    # initialize lists of bird information
    bird_data = {key : [] for key in key_list}
    wav_list = glob(bird_folder + '/wavs/*.wav')
    print(bird_name, len(wav_list))
    
    if visualize==True: wav_list = wav_list[:nex]

    # Syllabify/create dataset
    if parallel:
        with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
            bird_data_packed = parallel(
                delayed(w2s.process_bout)(wav_file,_mel_basis,hparams=hparams,submode=True, visualize = visualize) 
                     for wav_file in tqdm(wav_list))
    else:
        bird_data_packed = [w2s.process_bout(wav_file,_mel_basis, hparams=hparams,submode=True, visualize = visualize) for wav_file in tqdm(wav_list)]
    
    if np.sum([i[0] != [] for i in bird_data_packed]) == 0:
        print('Bird had no good bouts')
        continue
        
    for dtype, darray in zip(key_list, list(zip(*bird_data_packed))):
            [bird_data[dtype].extend(element) for element in darray] # flatten and clear darray -> bird_data[dtype]
            bird_data[dtype] = np.array(bird_data[dtype])

    # reformat bird syllables
    print('len dataset: ', len(bird_data['all_bird_syll_lengths']))
    
    # embed
    x = bird_data['all_bird_syll'].reshape((len(bird_data['all_bird_syll']), syll_size,syll_size))

    x_small = [resize(i, [16,16]) for i in tqdm(x)]
    x_small = np.array(x_small).reshape((len(x_small), np.prod(np.shape(x_small)[1:])))
    x_small = [(i*255).astype('uint8') for i in x_small]

    z = umap.UMAP(
        n_neighbors=30,
        #min_dist=0.0,
        n_components=2,
        random_state=42,
    ).fit_transform(x_small)
    #label
    labels = cluster_data(z,
          hdbscan.HDBSCAN,
          (),
          {'min_cluster_size':100,  'min_samples':1},
         verbose = True)
    # plot clusters
    plot_with_labels(np.array(list(z)), labels, figsize=(20,20))
    plt.show()

    if save:
        w2s.save_dataset(output_filename, 
                     bird_data['all_bird_syll'], 
                     bird_data['all_bird_syll_start'].astype('object'),
                     bird_data['all_bird_syll_lengths'], 
                     bird_data['all_bird_wav_file'].astype('object'),
                     bird_data['all_bird_t_rel_to_file'],
                     bird_name
                    )

